# <center> 1. Знакомимся с данными</center>

✍ Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов JOIN.

? А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

Обратите внимание! Вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.
Но прежде, как всегда, разберёмся с датасетом ↓

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим.

Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](data/dst3-u2-md4_1_1.jpg)

Таблица city — это справочник городов. Структура справочника представлена ниже.

|Название поля	|Тип данных	|Описание|
|---|---|---|
|city_id	|integer	|уникальный идентификатор города, первичный ключ|
|city_name	|text	|название города|
|state	|text	|штат, к которому относится город|
|population	|integer	|население города|
|area	|numeric	|площадь города|

Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

|Название поля	|Тип данных	|Описание|
|---|---|---|
|driver_id	|integer	|уникальный идентификатор водителя, первичный ключ|
|first_name	|text	|имя водителя|
|last_name	|text	|фамилия водителя|
|address	|text	|адрес водителя|
|zip_code	|integer	|почтовый индекс водителя|
|phone	|text	|телефон водителя|
|city_id	|integer	|идентификатор города водителя, внешний ключ к таблице city|

В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

|Название поля	|Тип данных	|Описание|
|---|---|---|
|truck_id	|integer	|Уникальный идентификатор грузовика, первичный ключ|
|make	|text	|Производитель грузовика|
|model_year	|integer	|Дата выпуска грузовика|

Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

|Название поля	|Тип данных	|Описание|
|---|---|---|
|ship_id	|integer	|уникальный идентификатор доставки, первичный ключ|
|cust_id	|integer	|идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer|
|weight	|numeric	|вес посылки|
|truck_id	|integer	|идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck|
|driver_id	|integer	|идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver|
|city_id	|integer	|идентификатор города в который совершена доставка, внешний ключ к таблице city|
|ship_date	|date	|дата доставки|

✍ Исследуйте датасет самостоятельно и ответьте на вопросы ниже ↓

→ Укажите название города с максимальным весом единичной доставки.

```sql
SELECT city_name, MAX(weight)
FROM sql.city c
JOIN sql.shipment s ON c.city_id=s.city_id
GROUP BY city_name
ORDER BY MAX(weight) DESC
LIMIT 1
```

→ Сколько различных производителей грузовиков перечислено в таблице truck?

```sql
SELECT COUNT(DISTINCT make) FROM sql.truck
```

→ Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

```sql
SELECT dr.first_name, sh.cust_id, COUNT(sh.ship_id)
FROM sql.shipment sh
JOIN sql.driver dr ON  dr.driver_id=sh.driver_id
GROUP BY dr.first_name, sh.cust_id
ORDER BY COUNT(sh.ship_id) DESC, dr.first_name
```

→ Укажите даты первой и последней по времени доставок в таблице shipment.

```sql
SELECT MAX(ship_date), MIN(ship_date)
FROM sql.shipment
```

→ Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.

```sql
SELECT cust.cust_name, COUNT(sh.ship_id)
FROM sql.customer AS cust
JOIN sql.shipment AS sh ON cust.cust_id=sh.cust_id
GROUP BY cust.cust_name
ORDER BY COUNT(sh.ship_id) DESC
LIMIT 1
```

# <center>2. UNION</center>

## <center>Принцип и условия работы Union</center>

✍ Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?

Чтобы разобраться в этом вопросе, смоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

```sql
SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name, задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/
FROM public.books /*из схемы public и таблицы books*/
UNION ALL /*оператор присоединения*/
SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/
FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/
```

Визуально произведённое нами действие можно представить следующим образом:  
![](data/dst3-u2-md4_2_1.png)

Общий принцип мы поняли, разберёмся в деталях:

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:  
* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

## <center>Виды UNION</center>

Оператор присоединения существует в двух вариантах:  
* UNION выводит только уникальные записи;
* UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

**Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.
Проверить, как различаются операторы, вы сможете при выполнении заданий 2.1 и 3.1.**

Синтаксис

Запрос строится таким образом:
```sql
SELECT         n columns
FROM 
         table_1
UNION ALL
SELECT 
         n columns
FROM 
         table_2
...
UNION ALL
SELECT 
         n columns
FROM 
         table_n
```
Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.  
Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

ажно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.
Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.  
```sql
SELECT
         c.city_id object_name,  'id города' object_type /*выбираем колонку city_id и задаём ей алиас object_name, сами задаём объект 'id города' и название столбца object_type*/
FROM 
         sql.city c /*из схемы sql и таблицы city, задаём алиас таблице — с*/
UNION ALL /*оператор присоединения*/
SELECT
         d.driver_id other_name,  'id водителя' other_type /*выбираем колонку driver_id и задаём ей алиас other_name, сами задаём объект 'id водителя' и название столбца other_type*/
FROM 
         sql.driver d  /*из схемы sql и таблицы driver, задаём алиас таблице — d*/
UNION ALL /*оператор присоединения*/
SELECT
         s.ship_id,  'id доставки' /*выбираем колонку ship_id, сами задаём объект 'id доставки'*/
FROM 
         sql.shipment s /*из схемы sql и таблицы shipment, задаём алиас таблице — s*/
UNION ALL /*оператор присоединения*/
SELECT
         c.cust_id,  'id клиента' /*выбираем колонку cust_id, сами задаём объект 'id клиента'*/
FROM 
         sql.customer c /*из схемы sql и таблицы customer, задаём алиас таблице — c*/
UNION ALL /*оператор присоединения*/
SELECT
         t.truck_id,  'id грузовика' /*выбираем колонку truck_id, сами задаём объект 'id грузовика'*/
FROM 
         sql.truck t /*из схемы sql и таблицы truck, задаём алиас таблице — t*/
ORDER BY 1 /*сортировка по первому столбцу*/
```

Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.  
Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.  
В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.  
Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.  
Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

```sql
SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
```

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.  
А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.  
Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!  
Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

```sql
SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
```

Вместо результата получим сообщение о синтаксической ошибке: "...syntax error at or near "UNION"..." Очевидно, этот фокус не удался.  
Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

```sql
(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)
```  
Отлично! Мы получили именно то, что хотели.

Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

```sql
SELECT c.city_name "название", 'city' "тип объекта"
FROM sql.city c
UNION
SELECT c.state, 'state'
FROM sql.city c
UNION
SELECT d.first_name, 'driver'
FROM sql.driver d
UNION
SELECT t.make, 'truck'
FROM sql.truck t
ORDER BY 1,2
```

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

```sql
SELECT c.city_name object_name
FROM sql.city c
UNION ALL
SELECT c.state
FROM sql.city c
ORDER BY 1
```

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

```sql
SELECT c.city_name object_name
FROM sql.city c
UNION
SELECT c.state
FROM sql.city c
ORDER BY 1
```

# <center> 3. UNION и ограничение типов данных</center>

## <center>Почему так важен тип данных?</center>

Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.  
Допустим, мы хотим вывести список всех id городов и их названий в одном столбце.